In [12]:
import numpy as np
import pandas as pd
import plotly.express as px

import yfinance as yf

indices = ['^GSPC', '^GDAXI', '^FCHI', '^N225','^FTSE']

data = yf.download(indices, start="2000-01-01", end="2024-10-01")['Adj Close']

# cut nan values
data = data.dropna()

# normalize data
data = data / data.iloc[0]

# risk free rate
symbol = '^FVX'
rf = yf.download(symbol, start="2000-01-01", end="2024-10-01")['Adj Close']
rf = rf.dropna() / 100 + 1
rf2 = rf.copy()
rf2 = rf2 - 0.01
rf2 = rf2**(1/365)
rf = rf**(1/365)

# cumulative prod
rf = rf.cumprod()
rf.name = 'rf'
rf2 = rf2.cumprod()
rf2.name = 'rf bank'

# concat rf with data
data = pd.concat([data, rf, rf2], axis=1).ffill()

[*********************100%%**********************]  5 of 5 completed
[*********************100%%**********************]  1 of 1 completed


In [13]:
fig = px.line(data, title='Indices performance', labels={'value':'Normalized price', 'Date':'Date'})
# change size
fig.update_layout(width=1000, height=600)

In [6]:
# Generate a list of dates for every week in 2024
weekly_dates_2024 = pd.date_range(start="2024-01-01", end="2024-12-31", freq='W')

initial_investment = 1000
steady_investment = 100

# create list of investments
steady_investments = [steady_investment] * len(weekly_dates_2024)
initial_investments = [initial_investment/10] * 10 + [0] * (len(weekly_dates_2024) - 10)

# create dataframe
df = pd.DataFrame({'Date': weekly_dates_2024, 'Steady Investment': steady_investments, 'Initial Investment': initial_investments})

# create stacked bar chart with plotly
fig = px.bar(df, x='Date', y=['Steady Investment', 'Initial Investment'], title='Investment over time', labels={'value': 'Investment', 'variable': 'Investment Type'}, barmode='stack')
fig.show()

In [11]:
fig = px.bar(df, x='Date', y=['Steady Investment', 'Initial Investment'], title='Investment over time', labels={'value': 'Investment', 'variable': 'Investment Type'}, barmode='stack')
fig.update_layout(width=1000, height=300)
fig.show()

In [38]:
def get_investment(initial_investment_, monthly_investment_, yearly_return_, total_years, year_setoff=0):
    plot_data = []
    monthly_return_ = yearly_return**(1/12)
    for years in range(total_years):
        payments = [monthly_investment_] * 12 * years
        investmens = monthly_return_ ** np.arange(0, 12 * years)
        value = np.sum(payments * investmens) + initial_investment_ * yearly_return_ ** years
        plot_data.append({'Years': years+year_setoff, 'Steady Investment': value})
    return pd.DataFrame(plot_data).set_index('Years')

df11 = get_investment(0, 100, 1.05, 10)
df12 = get_investment(0, 100, 1.08, 10)
df1 = df11 + df12
initial_investment = df1.iloc[-1]['Steady Investment']
df2 = get_investment(initial_investment, 150, 1.08, 30, 9)
df = pd.concat([df1, df2], axis=0).reset_index()

# plot
fig = px.line(df, x='Years', y='Steady Investment', title='Investment over time', labels={'value': 'Investment', 'variable': 'Investment Type'})
fig.update_layout(width=1000, height=600)
